In [ ]:
import requests
import pandas as pd
import time

# API Key OpenWeatherMap
api_key = "YOUR_API_KEY_HERE"

# List of cities
villes = [
    "Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg",
    "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon",
    "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes",
    "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban",
    "Biarritz", "Bayonne", "La Rochelle"
]

# URL API Nominatim (OpenStreetMap - latitude et longitude)
nominatim_url = "https://nominatim.openstreetmap.org/search"

# List to store Lat & Long
meteo_resultats = []

# loop to get all results for each city
for ville in villes:
    params = {
        "city": ville,
        "country": "France",
        "format": "json",
        "limit": 1
    }

    headers = {
        "User-Agent": "NotNecessary"
    }

    # API request
    r = requests.get(nominatim_url, params=params, headers=headers)

    if r.status_code == 200:
        data = r.json()
        if data:
            lat = data[0]["lat"]
            lon = data[0]["lon"]

            # print of the result
            print(f"Coordinate of {ville}: Latitude = {lat}, Longitude = {lon}")

            
            # API request to get weather forecast from coordinate
            weather_url = f"https://api.openweathermap.org/data/2.5/forecast"
            weather_params = {
                "lat": lat,
                "lon": lon,
                "units": "metric",
                "appid": api_key                
            }


            # Weather API request
            weather_r = requests.get(weather_url, params=weather_params)

            if weather_r.status_code == 200:
                weather_data = weather_r.json()

                # Store the result into a DataFrame

                for day in weather_data['list']:
                    meteo_resultats.append({
                        "City": ville,
                        "Latitude": lat,
                        "Longitude": lon,
                        "Date": pd.to_datetime(day['dt'], unit='s').strftime('%Y-%m-%d'),
                        "Temp_Max": day['main']['temp_max'],
                        "Temp_Min": day['main']['temp_min'],
                        "Humidity": day['main']['humidity'],
                        "Weather": day['weather'][0]['description']
                    })
            else:
                print(f"Weather ERRROR for {ville}: {weather_r.status_code}")
                meteo_resultats.append({
                    "City": ville,
                    "Latitude": lat,
                    "Longitude": lon,
                    "Error_Meteo": "Requête météo échouée"
                })
            
            
            time.sleep(1)
        else:
            print(f"No Data found for {ville}")
            meteo_resultats.append({
                "City": ville,
                "Coordinate_Error": "No result found"
            })
    else:
        print(f"Coordinates Error for {ville}: {r.status_code}")
        meteo_resultats.append({
            "City": ville,
            "Coordinate_Error": "Request Failed"
        })

# Convert into a DataFrame
df_meteo = pd.DataFrame(meteo_resultats)


Coordinate of Mont Saint Michel: Latitude = 48.6359541, Longitude = -1.511459954959514
Coordinate of St Malo: Latitude = 48.649518, Longitude = -2.0260409
Coordinate of Bayeux: Latitude = 49.2764624, Longitude = -0.7024738
Coordinate of Le Havre: Latitude = 49.4938975, Longitude = 0.1079732
Coordinate of Rouen: Latitude = 49.4404591, Longitude = 1.0939658
Coordinate of Paris: Latitude = 48.8534951, Longitude = 2.3483915
Coordinate of Amiens: Latitude = 49.8941708, Longitude = 2.2956951
Coordinate of Lille: Latitude = 50.6365654, Longitude = 3.0635282
Coordinate of Strasbourg: Latitude = 48.584614, Longitude = 7.7507127
Coordinate of Chateau du Haut Koenigsbourg: Latitude = 48.2495226, Longitude = 7.3454923
Coordinate of Colmar: Latitude = 48.0777517, Longitude = 7.3579641
Coordinate of Eguisheim: Latitude = 48.0447968, Longitude = 7.3079618
Coordinate of Besancon: Latitude = 47.2380222, Longitude = 6.0243622
Coordinate of Dijon: Latitude = 47.3215806, Longitude = 5.0414701
Coordinate o

In [4]:
display(df_meteo)

,City,Latitude,Longitude,Date,Temp_Max,Temp_Min,Humidity,Weather
0,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-16,9.08,6.89,79,clear sky
1,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-16,9.29,8.85,75,scattered clouds
2,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-17,9.51,9.51,75,broken clouds
3,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-17,10.10,10.10,75,overcast clouds
4,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-17,10.62,10.62,74,overcast clouds
...,...,...,...,...,...,...,...,...
1395,La Rochelle,46.159732,-1.1515951,2024-11-21,12.71,12.71,88,moderate rain
1396,La Rochelle,46.159732,-1.1515951,2024-11-21,12.28,12.28,76,light rain
1397,La Rochelle,46.159732,-1.1515951,2024-11-21,10.76,10.76,94,light rain
1398,La Rochelle,46.159732,-1.1515951,2024-11-21,13.70,13.70,91,moderate rain


In [5]:
df_meteo_paris = df_meteo[df_meteo["City"] == "Paris"]
display(df_meteo_paris)

,City,Latitude,Longitude,Date,Temp_Max,Temp_Min,Humidity,Weather
200,Paris,48.8534951,2.3483915,2024-11-16,10.48,9.35,70,broken clouds
201,Paris,48.8534951,2.3483915,2024-11-16,9.28,9.13,67,broken clouds
202,Paris,48.8534951,2.3483915,2024-11-16,8.45,8.00,70,scattered clouds
203,Paris,48.8534951,2.3483915,2024-11-17,7.10,7.10,75,few clouds
204,Paris,48.8534951,2.3483915,2024-11-17,7.98,7.98,73,broken clouds
205,Paris,48.8534951,2.3483915,2024-11-17,8.41,8.41,72,overcast clouds
206,Paris,48.8534951,2.3483915,2024-11-17,8.52,8.52,78,light rain
207,Paris,48.8534951,2.3483915,2024-11-17,10.54,10.54,72,overcast clouds
208,Paris,48.8534951,2.3483915,2024-11-17,10.99,10.99,60,overcast clouds
209,Paris,48.8534951,2.3483915,2024-11-17,9.62,9.62,67,overcast clouds


In [6]:
df_meteo.head()

,City,Latitude,Longitude,Date,Temp_Max,Temp_Min,Humidity,Weather
0,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-16,9.08,6.89,79,clear sky
1,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-16,9.29,8.85,75,scattered clouds
2,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-17,9.51,9.51,75,broken clouds
3,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-17,10.10,10.10,75,overcast clouds
4,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-17,10.62,10.62,74,overcast clouds


In [7]:
df_meteo["Weather"].unique()

array(['clear sky', 'scattered clouds', 'broken clouds',
       'overcast clouds', 'light rain', 'moderate rain', 'light snow',
       'few clouds', 'snow', 'heavy intensity rain'], dtype=object)

In [8]:
# Dictionary for the quotation of weather condition, good condition get positive number
# and bad condition negative.
notations = {
    "clear sky": 600,
    "few clouds": 500,
    "scattered clouds": 400,
    "broken clouds": 300,
    "overcast clouds": 200,    

    "light intensity drizzle": -1,
    "drizzle": -2,
    "heavy intensity drizzle": -3,
    "light intensity drizzle rain": -4,
    "drizzle rain": -5,
    "heavy intensity drizzle rain": -6,
    "shower drizzle": -7,
    "shower rain and drizzle": -8,
    "heavy shower rain and drizzle": -9,    

    "light rain": -10,
    "moderate rain": -20,
    "heavy intensity rain": -30,
    "very heavy rain": -40,
    "extreme rain": -50,
    "freezing rain": -60,
    "light intensity shower rain": -70,
    "shower rain": -80,
    "heavy intensity shower rain": -90,
    "ragged shower rain": -100,

    "thunderstorm with light drizzle": -15,
    "thunderstorm with drizzle": -25,
    "thunderstorm with light rain": -35,
    "thunderstorm with rain": -45,
    "thunderstorm with heavy drizzle": -55,
    "thunderstorm with heavy rain": -65,
    "thunderstorm": -75,
    "heavy thunderstorm": -85,
    "ragged thunderstorm": -95,

    "light snow": -20,
    "snow": -40,
    "heavy snow": -60,
    "sleet": -80,
    "light shower sleet": -100,
    "shower sleet": -120,
    "light rain and snow": -140,
    "rain and snow": -160,
    "light shower snow": -180,
    "shower snow": -200,
    "heavy shower snow": -220,

    "mist": -10,
    "smoke": -20,
    "haze": -30,
    "sand/dust whirls": -40,
    "fog": -50,
    "sand": -60,
    "dust": -70,
    "volcanic ash": -90,
    "squalls": -100,
    "tornado": -400,
}


In [9]:
df_meteo["Weather_Score"] = df_meteo["Weather"].map(notations)

display(df_meteo)

,City,Latitude,Longitude,Date,Temp_Max,Temp_Min,Humidity,Weather,Weather_Score
0,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-16,9.08,6.89,79,clear sky,600
1,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-16,9.29,8.85,75,scattered clouds,400
2,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-17,9.51,9.51,75,broken clouds,300
3,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-17,10.10,10.10,75,overcast clouds,200
4,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-17,10.62,10.62,74,overcast clouds,200
...,...,...,...,...,...,...,...,...,...
1395,La Rochelle,46.159732,-1.1515951,2024-11-21,12.71,12.71,88,moderate rain,-20
1396,La Rochelle,46.159732,-1.1515951,2024-11-21,12.28,12.28,76,light rain,-10
1397,La Rochelle,46.159732,-1.1515951,2024-11-21,10.76,10.76,94,light rain,-10
1398,La Rochelle,46.159732,-1.1515951,2024-11-21,13.70,13.70,91,moderate rain,-20


In [10]:
# Total "Weather Score" points per city
total_points_par_ville = df_meteo.groupby("City")["Weather_Score"].sum().reset_index()
total_points_par_ville = total_points_par_ville.sort_values(by="Weather_Score", ascending=False)

display(total_points_par_ville)

,City,Weather_Score
10,Bormes les Mimosas,12660
18,Gorges du Verdon,12430
12,Cassis,11840
1,Aix en Provence,11640
24,Marseille,11620
0,Aigues Mortes,11350
33,Toulouse,11250
14,Collioure,11190
11,Carcassonne,10960
27,Nimes,10840


In [11]:
df_meteo["Temp_Avg"] = (df_meteo["Temp_Max"] + df_meteo["Temp_Min"]) / 2
df_meteo.head()

,City,Latitude,Longitude,Date,Temp_Max,Temp_Min,Humidity,Weather,Weather_Score,Temp_Avg
0,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-16,9.08,6.89,79,clear sky,600,7.985
1,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-16,9.29,8.85,75,scattered clouds,400,9.070
2,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-17,9.51,9.51,75,broken clouds,300,9.510
3,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-17,10.10,10.10,75,overcast clouds,200,10.100
4,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-17,10.62,10.62,74,overcast clouds,200,10.620


In [12]:
import datetime

today_date = datetime.datetime.now().strftime("%Y-%m-%d")
name_file = f"weather_project_{today_date}.csv"

df_meteo.to_csv(name_file, index=True)

In [13]:
# Grouped by "City" and "Date" to get the weather score and the average temperature
df_grouped = df_meteo.groupby(['City', 'Date']).agg({
    'Weather_Score': 'sum',
    'Temp_Avg': 'mean'
}).unstack(level='Date')

# Display the AVG temp and Weather score side by side for each date
df_grouped = df_grouped.swaplevel(axis=1).sort_index(axis=1).round(2)

display(df_grouped)

Date                         2024-11-16               2024-11-17  \
                               Temp_Avg Weather_Score   Temp_Avg   
City                                                               
Aigues Mortes                     12.00           800      12.99   
Aix en Provence                   12.75           -20      12.79   
Amiens                             8.13           700       8.56   
Annecy                             6.80           900       7.20   
Ariege                            13.55           700      13.00   
Avignon                           11.61           400      11.35   
Bayeux                             9.81           500       9.59   
Bayonne                           15.00           500      13.25   
Besancon                           5.24           800       5.46   
Biarritz                          15.47           500      13.70   
Bormes les Mimosas                13.60           600      14.17   
Carcassonne                       11.00          1000      12.64   
Cassis                            14.89           -20      14.73   
Chateau du Haut Koenigsbourg       1.86           500       3.74   
Collioure                         13.89           500      14.95   
Colmar                             4.76           500       6.70   
Dijon                              5.24           400       6.05   
Eguisheim                          5.62           800       6.54   
Gorges du Verdon                   6.17          1100       6.90   
Grenoble                           8.56           900       9.24   
La Rochelle                       11.42          1000       9.62   
Le Havre                          11.15           190      11.68   
Lille                              9.36           400       8.72   
Lyon                               8.74           700       8.37   
Marseille                         15.42           -20      15.39   
Mont Saint Michel                  8.53          1000      10.46   
Montauban                         12.32          1000      10.69   
Nimes                             10.40           500      11.53   
Paris                              9.12          1000       8.99   
Rouen                              7.42           500       9.10   
Saintes Maries de la mer          13.64           -20      13.51   
St Malo                           10.64           400      11.57   
Strasbourg                         4.47           500       5.58   
Toulouse                          13.63          1600      12.19   
Uzes                               9.84           500      10.75   

Date                                       2024-11-18                \
                             Weather_Score   Temp_Avg Weather_Score   
City                                                                  
Aigues Mortes                         3000      12.20          2800   
Aix en Provence                       2900      10.44          2400   
Amiens                                1180       9.20           330   
Annecy                                3200       5.76          2300   
Ariege                                2800      11.17          1800   
Avignon                               2600      10.76          2300   
Bayeux                                1180      10.42           330   
Bayonne                               1790      12.45          1600   
Besancon                              2000       7.15          1600   
Biarritz                              1370      12.74          1600   
Bormes les Mimosas                    3300      12.39          2500   
Carcassonne                           2800      10.96          2300   
Cassis                                2390      13.06          2500   
Chateau du Haut Koenigsbourg          1590       5.21          1180   
Collioure                             2700      14.12          2800   
Colmar                                1490       7.87          1600   
Dijon                                 2100       7.39          160

In [14]:
df_meteo.head()

,City,Latitude,Longitude,Date,Temp_Max,Temp_Min,Humidity,Weather,Weather_Score,Temp_Avg
0,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-16,9.08,6.89,79,clear sky,600,7.985
1,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-16,9.29,8.85,75,scattered clouds,400,9.070
2,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-17,9.51,9.51,75,broken clouds,300,9.510
3,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-17,10.10,10.10,75,overcast clouds,200,10.100
4,Mont Saint Michel,48.6359541,-1.511459954959514,2024-11-17,10.62,10.62,74,overcast clouds,200,10.620


In [15]:
import plotly.express as px

fig = px.scatter(df_meteo, x = "Weather_Score", y = "Temp_Avg")
fig.show()

In [19]:
df_meteo["Latitude"] = pd.to_numeric(df_meteo["Latitude"], errors="coerce")
df_meteo["Longitude"] = pd.to_numeric(df_meteo["Longitude"], errors="coerce")

df_meteo_pos = df_meteo[df_meteo["Weather_Score"] >= 0]

In [20]:
fig = px.density_mapbox(
    df_meteo_pos,
    lat="Latitude",
    lon="Longitude",
    z="Temp_Avg",
    mapbox_style="open-street-map",
    animation_frame="Date",
    zoom=3.5,
    radius=10,
    center={"lat": 46.603354, "lon": 1.888334}
)

fig.show()